In [4]:
import random

class DiffieHellman:
    
    def __init__(self, key_size: int = 512):
        self.key_size = key_size
        self.p = None           # Số nguyên tố chung (Prime)
        self.g = None           # Cơ số chung (Generator)
        self.__private_key = None # Khóa bí mật (a hoặc b) - Private Attribute
        self.public_key = None    # Khóa công khai (A hoặc B)
        self.shared_secret = None # Khóa chung thống nhất (s)

    def _modular_exponentiation(self, base: int, exp: int, mod: int) -> int:

        result = 1
        base = base % mod
        while exp > 0:
            if exp % 2 == 1:
                result = (result * base) % mod
            base = (base * base) % mod
            exp = exp // 2
        return result

    def _is_prime_miller_rabin(self, n: int, k: int = 40) -> bool:

        if n == 2 or n == 3: return True
        if n % 2 == 0 or n < 2: return False

        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        
        for _ in range(k):
            a = random.randint(2, n - 2)
            x = self._modular_exponentiation(a, d, n)
            if x == 1 or x == n - 1:
                continue
            for _ in range(r - 1):
                x = self._modular_exponentiation(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        return True

    def _generate_prime(self, bits: int) -> int:
        """Sinh số nguyên tố n-bit"""
        while True:
            num = random.getrandbits(bits)
            num |= (1 << (bits - 1)) | 1 
            if self._is_prime_miller_rabin(num):
                return num


    def generate_parameters(self):
     
        print(f"[*] Đang đào số nguyên tố {self.key_size} ")
        
        # 1. Sinh số nguyên tố p
        self.p = self._generate_prime(self.key_size)
        
        # 2. Chọn số sinh g (Generator)
        # Trong thực tế cần chọn g là căn nguyên thủy, 
        # nhưng với mục đích học, g ngẫu nhiên là đủ để thuật toán chạy đúng.
        self.g = random.randint(2, self.p - 2)
        
        print(f" Đã sinh tham số chung:")
        print(f"    p = {str(self.p)[:15]}")
        print(f"    g = {str(self.g)[:15]}")

    def set_parameters(self, p: int, g: int):

        self.p = p
        self.g = g

    def generate_private_key(self):

        if not self.p: 
            raise Exception("Lỗi: Chưa có tham số p, g!")
            
        self.__private_key = random.randint(2, self.p - 2)
        print("Đã sinh khóa Bí Mật")

    def calculate_public_key(self) -> int:

        if not self.__private_key: 
            raise Exception("Lỗi: Chưa có Private Key!")
            
        self.public_key = self._modular_exponentiation(self.g, self.__private_key, self.p)
        return self.public_key

    def calculate_shared_secret(self, other_public_key: int) -> int:

        if not self.__private_key: 
            raise Exception("Lỗi: Chưa có Private Key!")
            
        self.shared_secret = self._modular_exponentiation(other_public_key, self.__private_key, self.p)
        return self.shared_secret

# --- PHẦN 3: TEST DRIVE (MÔ PHỎNG ALICE & BOB) ---
if __name__ == "__main__":
    
    # 1. Khởi tạo 2 người (Key 512 bit cho nhanh)
    alice = DiffieHellman(key_size=512)
    bob = DiffieHellman(key_size=512)
    
    # --- GIAI ĐOẠN 1: THIẾT LẬP ---
    alice.generate_parameters()
    
    # Alice gửi p, g cho Bob qua mạng
    bob.set_parameters(alice.p, alice.g)
    
    # --- GIAI ĐOẠN 2: BÍ MẬT ---
    alice.generate_private_key() # Alice chọn a
    bob.generate_private_key()   # Bob chọn b
    
    # --- GIAI ĐOẠN 3: CÔNG KHAI ---
    alice_pub = alice.calculate_public_key() # A = g^a mod p
    bob_pub = bob.calculate_public_key()     # B = g^b mod p
    
    print(f"[LINH] Public Key (A): {str(alice_pub)[:15]}...")
    print(f"[KHOA]   Public Key (B): {str(bob_pub)[:15]}...")
    
    # Alice dùng B của Bob
    alice_s = alice.calculate_shared_secret(bob_pub)
    
    # Bob dùng A của Alice
    bob_s = bob.calculate_shared_secret(alice_pub)
    
    print(f"[LINH] Tính ra s: {str(alice_s)[:20]}...")
    print(f"[KHOA]   Tính ra s: {str(bob_s)[:20]}...")
    
    # KIỂM TRA
    if alice_s == bob_s:
        print("\n=> THÀNH CÔNG! Hai bên đã có chung một bí mật. ")
        print(f"=> Độ dài khóa chung: {alice_s.bit_length()} bits")
    else:
        print("\n=> THẤT BẠI!. ")

[*] Đang đào số nguyên tố 512 
 Đã sinh tham số chung:
    p = 107544609465852
    g = 103447255446094
Đã sinh khóa Bí Mật
Đã sinh khóa Bí Mật
[LINH] Public Key (A): 598169908857799...
[KHOA]   Public Key (B): 185339821959485...
[LINH] Tính ra s: 93376463331932784232...
[KHOA]   Tính ra s: 93376463331932784232...

=> THÀNH CÔNG! Hai bên đã có chung một bí mật. 
=> Độ dài khóa chung: 512 bits
